In [1]:
import cv2
import json
import os
import pprint
import requests

import matplotlib.pyplot as plt
import numpy as np

from azure_cfg import cv_key
from skimage import io

%matplotlib inline

In [2]:
# set up global variables
subscription_key = cv_key

vision_base_url = "https://westcentralus.api.cognitive.microsoft.com/vision/v1.0/"
vision_analyze_url = vision_base_url + "analyze?"

image_dir = '../images/'
train_image_dir = '../images/train/'
test_image_dir = '../images/test/'

In [3]:
# evaluate an image using the Microsoft Azure Cognitive Services Computer Vision API
def extract_high_level_features(image_path):
    headers  = {'Ocp-Apim-Subscription-Key': subscription_key,
                'Content-Type'             : 'application/octet-stream'}
    params   = {'visualFeatures': 'Adult,Categories,Description,Color,Faces,ImageType,Tags'}
    body     = open(image_path, 'rb')

    response = requests.post(vision_analyze_url, headers=headers, params=params, data=body)
    response.raise_for_status()
    data = response.json()
    
    features = {
        "clip_art_type"                      : data["imageType"]["clipArtType"],
        "line_drawing_type"                  : data["imageType"]["lineDrawingType"],
        "is_black_and_white"                 : int(data["color"]["isBwImg"]),
        "is_adult_content"                   : int(data["adult"]["isAdultContent"]),
        "adult_score"                        : data["adult"]["adultScore"],
        "is_racy"                            : int(data["adult"]["isRacyContent"]),
        "racy_score"                         : data["adult"]["racyScore"],
        "has_faces"                          : int(len(data["faces"])),
        "num_faces"                          : len(data["faces"]),
        "is_dominant_color_background_black" : int(data["color"]["dominantColorBackground"] == "Black"),
        "is_dominant_color_foreground_black" : int(data["color"]["dominantColorForeground"] == "Black")
    }
    
    return list(features.values())

In [4]:
# initialize HOG descriptor with (mostly) default supported parameters
width, height = 128, 64
descriptor = cv2.HOGDescriptor(_winSize = (width,height),
                               _blockSize = (16,16),
                               _blockStride = (8,8),
                               _cellSize = (8,8),
                               _nbins = 9)

# compute HOG features for an image
def extract_low_level_features(image_path, descriptor=descriptor):
    image = io.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (width,height))

    hog = descriptor.compute(image)
    hog = hog.flatten().tolist()
    return hog

In [5]:
# conduct low- and high-level feature extraction for images in a directory
def extract_all_features(image_dir):
    features = []
    for image_path in os.listdir(image_dir):
        path = image_dir + image_path

        high_level_features = extract_high_level_features(path)
        low_level_features = extract_low_level_features(path)
        features.append(high_level_features + low_level_features)
    
    return features

In [6]:
train_features = extract_all_features(train_image_dir)
test_features = extract_all_features(test_image_dir)
print('Train features:\n', train_features)
print('Test features:\n', test_features)

HTTPError: 429 Client Error: Too Many Requests for url: https://westcentralus.api.cognitive.microsoft.com/vision/v1.0/analyze?visualFeatures=Adult%2CCategories%2CDescription%2CColor%2CFaces%2CImageType%2CTags

In [ ]:
# get the class labels for the training instances
def get_class_labels():
    train_labels = json.load(open('../images/train.json'))
    
    train_classes = [label['answerable'] for label in train_labels]
    return train_classes

In [ ]:
train_classes = get_class_labels()
print('Train classes:\n', train_classes)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X=train_features, y=train_classes)

y_pred_rf = rnd_clf.predict(test_features)
print(y_pred_rf)